In [ ]:
import re
import json
from collections import defaultdict

file_name = "tcas0"

# Remove duplicates from a list while preserving the original order
def remove_duplicates_preserve_order(items):
    seen = set()
    result = []
    for item in items:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result

# Parse a GCC-style control flow graph (CFG) text into structured data
def parse_cfg(raw_text):
    functions = []
    current_function = None
    successors = defaultdict(list)
    basic_blocks = defaultdict(list)
    current_bb = None

    for line in raw_text.splitlines():
        # Detect the beginning of a new function
        func_match = re.match(r';; Function (\w+)', line)
        if func_match:
            if current_function:
                # Store the previously parsed function data
                functions.append((current_function, dict(successors), dict(basic_blocks)))
                successors.clear()
                basic_blocks.clear()
            current_function = func_match.group(1)
            current_bb = None

        # Extract basic block successors from lines like: ;; 1 succs { 2 3 }
        succ_match = re.match(r';;\s*(\d+)\s+succs\s+\{([^}]*)\}', line)
        if succ_match:
            from_bb = int(succ_match.group(1))
            to_bbs = list(map(int, succ_match.group(2).split()))
            successors[from_bb].extend(to_bbs)

        # Identify basic block headers such as: <bb 2>:
        bb_match = re.match(r'\s*<bb\s+(\d+)>', line)
        if bb_match:
            current_bb = int(bb_match.group(1))
            continue

        # Collect source code references inside each basic block
        if current_bb is not None:
            code_match = re.search(r'\[(\w+\.c:\d+):\d+(\d|\w|\s)*\]', line)
            # print(code_match, line)
            if code_match:
                basic_blocks[current_bb].append(code_match.group(1))

    # Add the last function if present
    if current_function:
        functions.append((current_function, dict(successors), dict(basic_blocks)))

    return functions

# Read the CFG text from a file and process it
with open(file_name + ".c.015t.cfg", "r") as f:
    cfg_text = f.read()

functions = parse_cfg(cfg_text)

In [ ]:
import json

output_file = file_name + "_cfg_all_functions.json"
all_graphs = []

for func_name, edges, blocks in functions:
    # 1. Deduplicate blocks by an *exact* set of lines.
    content_to_bb = {}
    bb_mapping = {}

    for bb, lines in blocks.items():
        key = frozenset(lines)
        if key not in content_to_bb:
            content_to_bb[key] = f"bb{bb}"
        bb_mapping[f"bb{bb}"] = content_to_bb[key]

    # Remap edges to basic block names using the deduplication mapping.
    edge_list = set()
    for src, dsts in edges.items():
        src_bb = bb_mapping.get(f"bb{src}", f"bb{src}")
        for dst in dsts:
            dst_bb = bb_mapping.get(f"bb{dst}", f"bbend")
            if src_bb != dst_bb:
                edge_list.add((src_bb, dst_bb))
                

    # Generate compressed nodes dictionary.
    node_dict = {}
    for lines_set, block_label in content_to_bb.items():
        node_dict[block_label] = {
            "lines": sorted(lines_set, key=lambda x: int(x.split(":")[1])),
        }

    # 1. Collect all unique basic block labels from edges and nodes.
    unique_blocks = set(['bbend'])
    for src, dst in edge_list:
        unique_blocks.add(src)
        unique_blocks.add(dst)
    unique_blocks.update(node_dict.keys())
    
    print(unique_blocks)

    # 3. Sort the remaining blocks by their numeric value.
    sorted_blocks = sorted(unique_blocks, key=lambda bb: int(bb[2:]) if bb[2:].isdigit() else float('inf'))

    rename = {
        sorted_blocks[i]: f"bb{i+1}" if sorted_blocks[i][2:].isdigit() else sorted_blocks[i]
        for i in range(len(sorted_blocks))
    }

    # 6. Update the edge list with the new names.
    edge_list = [[rename[src], rename[dst]] for src, dst in edge_list]

    # 7. Update the node dictionary:
    new_nodes = {}
    for bb in unique_blocks:
        new_nodes[rename[bb]] = node_dict.get(bb, {})
    node_dict = new_nodes
    
    cfg_json = {
        "function": func_name,
        "edges": sorted([list(edge) for edge in edge_list]),
        "nodes": node_dict
    }

    all_graphs.append(cfg_json)

with open(output_file, "w") as f:
    json.dump(all_graphs, f, indent=2)

print(f"Compressed CFGs exported to `{output_file}`")
